In [2]:
import jammy_flows
import h5py
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import math

In [3]:
class HDF5PhotonTable(torch.utils.data.IterableDataset):
    def __init__(self, filename, start, end):
        super(HDF5PhotonTable).__init__()
        assert end > start, "end >= start"
        self.start = start
        self.end = end
        self._hdl = h5py.File(filename)
        

    def __iter__(self):
        worker_info = torch.utils.data.get_worker_info()
        if worker_info is None:  # single-process data loading, return the full iterator
            iter_start = self.start
            iter_end = self.end
        else:  # in a worker process
            # split workload
            per_worker = int(math.ceil((self.end - self.start) / float(worker_info.num_workers)))
            worker_id = worker_info.id
            iter_start = self.start + worker_id * per_worker
            iter_end = min(iter_start + per_worker, self.end)

        

        return iter(range(iter_start, iter_end))

    def __del__(self):
        self._hdl.close()
        
        

In [4]:
with h5py.File('../assets/photon_table.hd5', 'r') as f:
    print(f["photon_tables"].keys())

#mask = f["photon_tables/dataset_1"][0] == 1



<KeysViewHDF5 ['dataset_1', 'dataset_10', 'dataset_100', 'dataset_1000', 'dataset_1001', 'dataset_1002', 'dataset_1003', 'dataset_1004', 'dataset_1005', 'dataset_1006', 'dataset_1007', 'dataset_1008', 'dataset_1009', 'dataset_101', 'dataset_1010', 'dataset_1011', 'dataset_1012', 'dataset_1013', 'dataset_1014', 'dataset_1015', 'dataset_1016', 'dataset_1017', 'dataset_1018', 'dataset_1019', 'dataset_102', 'dataset_1020', 'dataset_1021', 'dataset_1022', 'dataset_1023', 'dataset_1024', 'dataset_1025', 'dataset_1026', 'dataset_1027', 'dataset_1028', 'dataset_1029', 'dataset_103', 'dataset_1030', 'dataset_1031', 'dataset_1032', 'dataset_1033', 'dataset_1034', 'dataset_1035', 'dataset_1036', 'dataset_1037', 'dataset_1038', 'dataset_1039', 'dataset_104', 'dataset_1040', 'dataset_1041', 'dataset_1042', 'dataset_1043', 'dataset_1044', 'dataset_1045', 'dataset_1046', 'dataset_1047', 'dataset_1048', 'dataset_1049', 'dataset_105', 'dataset_1050', 'dataset_1051', 'dataset_1052', 'dataset_1053', 'dat

<KeysViewHDF5 ['dataset_1', 'dataset_10', 'dataset_11', 'dataset_12', 'dataset_2', 'dataset_3', 'dataset_4', 'dataset_5', 'dataset_6', 'dataset_7', 'dataset_8', 'dataset_9']>

In [6]:
f.close()